In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
df = pd.read_csv('universities-clean.csv')

In [3]:
df.head()

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [4]:
conn = psycopg2.connect(
    host="localhost",
    port=5435,
    user='Wikipediauser',
    password='Wikipediauserpassword',
    database="Wikipediadb"
)

In [5]:
cursor = conn.cursor()

In [6]:
create_query_file = open('./sql/postgres_create_table.sql')
create_query = create_query_file.read()
create_query

'CREATE TABLE university (\n    id SERIAL PRIMARY KEY,\n    country VARCHAR(255) NOT NULL,\n    name VARCHAR(255) NOT NULL,\n    founded INT NOT NULL,\n    type VARCHAR(255) NOT NULL,\n    enrollment BIGINT NOT NULL,\n    link VARCHAR(255) NOT NULL,\n    CONSTRAINT unique_combination UNIQUE (country, name)\n);'

In [7]:
cursor.execute(create_query)

In [8]:
conn.commit()

In [20]:
conn.rollback()

In [10]:
engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)

In [18]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link


In [21]:
df = pd.read_csv('universities-clean.csv')
data = list(df.itertuples(name=None, index=None))
merge_query_file = open("./sql/postgres_upsert.sql")
merge_query = merge_query_file.read()
merge_query

'INSERT INTO university (country, name, founded, type, enrollment, link)\nVALUES (%s, %s, %s, %s, %s, %s)\nON CONFLICT (country, name) DO UPDATE\nSET\n    founded=EXCLUDED.founded,\n    type=EXCLUDED.type,\n    enrollment=EXCLUDED.enrollment,\n    link=EXCLUDED.link;'

In [22]:
cursor.executemany(merge_query, data)

In [23]:
conn.commit()

In [24]:
pd.read_sql("SELECT * FROM university;", con=engine)

,id,country,name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
66,67,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
67,68,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
68,69,United States,Arizona State University,1885,Public,194000,https://en.wikipedia.org/wiki/Arizona_State_Un...
69,70,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
